# 합친것

In [153]:
def summary(user_text):
    yt = YouTube(user_text)
    caption = yt.captions.get_by_language_code('ko.7XP2tGORuV4')
    if caption == None:
        caption = yt.captions.all()[0]
    text = caption.xml_captions
    불용어 = ['<[^가-힣]+>', '&[^가-힣]+;', '(박수 소리)', '\n', '( )', ':' , '(앵커)', '(기자)']
    for i in 불용어:
        text = re.sub(i,' ', text)
        
    kkma = Kkma()

    # text입력받아 kkma.sentences()를 이용해 문장단위로 나는 뒤 sentences로 리턴
    def text2sentences(text):
        sentences= kkma.sentences(text)  # text일 때 문장별로 리스트 만듦
        for idx in range(0, len(sentences)): #길이에 따라 문장 합침
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    sentences = text2sentences(text)
    twitter = Twitter()

    #불용어제거
    stopwords = ['( )','안녕하세요', '아', '으', '네' ,'코']
    def get_nouns(sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                    if noun not in stopwords and len(noun) > 1]))
        return nouns
    nouns = get_nouns(sentences)
    tfidf = TfidfVectorizer()
    cnt_vec = CountVectorizer()
    graph_sentence = []
    def build_sent_graph(sentence):
        tfidf_mat = tfidf.fit_transform(sentence).toarray()
        graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return graph_sentence

    sent_graph = build_sent_graph(nouns)
    def build_words_graph(sentence):
        cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    words_graph, idx2word = build_words_graph(nouns)
    def get_ranks(graph, d=0.85):
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0
            link_sum = np.sum(A[:,id])
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B)
        return {idx: r[0] for idx, r in enumerate(ranks)}
    sent_rank_idx = get_ranks(sent_graph)
    sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)
    word_rank_idx = get_ranks(words_graph)
    sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)
    
    summarize = []
    index = []
    for idx in sorted_sent_rank_idx[:5]:
        index.append(idx)
        
    index.append(idx)
    
    for idx in index:
        summarize.append(sentences[idx])
        
    for text in summarize[:]:
        llist.append(text)
    
    m = list(set(llist))
    
    keywords = []
    index = []
    for idx in sorted_word_rank_idx[:15]:
        index.append(idx)
        
    for idx in index:
        keywords.append(idx2word[idx])
        
    keywords = [each_word for each_word in keywords
                if each_word not in stop_words]
        
    keyword = keywords[:5]
    return m, keyword

<>:29: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:29: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\82102\AppData\Local\Temp/ipykernel_18324/4064985180.py:29: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [154]:
from pytube import YouTube
import re
user_text = 'https://www.youtube.com/watch?v=BNlZsg5RPxM&list=PLUHG6IBxDr3i8GaZ6wur2VGjr8HJjbGaz&index=46'

In [155]:
n,key=summary(user_text)

C:\Users\82102\AppData\Local\Temp/ipykernel_18324/4064985180.py:3: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('ko.7XP2tGORuV4')
C:\Users\82102\anaconda3.1\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [156]:
n

[' 성장이냐',
 '',
 '지금은 물가 잡는 게 가장 급하기 때문에 환영 받지 않더라도 금리 인상 신호를 주며 할 일을 하겠다는 겁니다.',
 '-( ) 새 한국은행 총재 인사청문회에서 이 창용 후보자가 금리를 올리겠다는 신호를 강하게 보냈습니다.',
 '새 한국은행 총재 인사청문회에서 이 창용 후보자가 금리를 올리겠다는 신호를 강하게 보냈습니다.',
 '악역도 마다하지 않겠다고 했습니다.',
 '미국처럼 물가가 오른 뒤가 아닌 선제적으로 금리를 올려야 한다고 강조했습니다.',
 '지금까지 사회부 신용 식 기자였습니다.',
 '는 질문에 이 창용 한국은행 총재 후보자는 지금은 물가부터 잡아야 할 때라는 답을 내놨습니다.']

In [157]:
key

['물가', '금리', '정부', '정책', '인상']

# 분할

In [ ]:
def summary(user_text):
    yt = YouTube(user_text)
    caption = yt.captions.get_by_language_code('ko.7XP2tGORuV4')
    if caption == None:
        caption = yt.captions.all()[0]
    text = caption.xml_captions
    불용어 = ['<[^가-힣]+>', '&[^가-힣]+;', '(박수 소리)', '\n', '( )', ':' , '(앵커)', '(기자)']
    for i in 불용어:
        text = re.sub(i,' ', text)
        
    kkma = Kkma()

    # text입력받아 kkma.sentences()를 이용해 문장단위로 나는 뒤 sentences로 리턴
    def text2sentences(text):
        sentences= kkma.sentences(text)  # text일 때 문장별로 리스트 만듦
        for idx in range(0, len(sentences)): #길이에 따라 문장 합침
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    sentences = text2sentences(text)
    twitter = Twitter()

    #불용어제거
    stopwords = ['( )','안녕하세요', '아', '으', '네' ,'코']
    def get_nouns(sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                    if noun not in stopwords and len(noun) > 1]))
        return nouns
    nouns = get_nouns(sentences)
    tfidf = TfidfVectorizer()
    cnt_vec = CountVectorizer()
    graph_sentence = []
    def build_sent_graph(sentence):
        tfidf_mat = tfidf.fit_transform(sentence).toarray()
        graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return graph_sentence

    sent_graph = build_sent_graph(nouns)
    def build_words_graph(sentence):
        cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    words_graph, idx2word = build_words_graph(nouns)
    def get_ranks(graph, d=0.85):
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0
            link_sum = np.sum(A[:,id])
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B)
        return {idx: r[0] for idx, r in enumerate(ranks)}
    sent_rank_idx = get_ranks(sent_graph)
    sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)
    word_rank_idx = get_ranks(words_graph)
    sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)
    
    summarize = []
    index = []
    for idx in sorted_sent_rank_idx[:5]:
        index.append(idx)
        
    index.append(idx)
    
    for idx in index:
        summarize.append(sentences[idx])
        
    for text in summarize[:]:
        llist.append(text)
    
    m = list(set(llist))
    
    keywords = []
    index = []
    for idx in sorted_word_rank_idx[:15]:
        index.append(idx)
        
    for idx in index:
        keywords.append(idx2word[idx])
        
    keywords = [each_word for each_word in keywords
                if each_word not in stop_words]
        
    keyword = keywords[:5]
    return m, keyword

In [11]:
with open('C:/Users/82102/Desktop/4월 프로젝트/total_texts.txt', 'r', encoding='UTF8') as file: 
    a = file.readlines()
a

['구매 개수를 제한할 정도로 지금 그야말로 식용유 대란입니다. 치킨은 뭘로 튀기나 자영업자 분들의 하소연도 이어지고 있는데요. 어려운 사정은 국내뿐만이 아닙니다. 지금 전 세계가 그야말로 식용유 보릿고개를 넘고 있습니다. 앞서 스페인 벨기에 등 유럽 각국의 대형마트에서도 식용유의 구매 개수를 제한한 바 있는데요. 특히 영국의 대표적인 튀김 체인점인 피시인 칩스가 3천 곳 넘게 문을 닫았고 영국 정부까지 나서 체인점 폐점 위기를 경고할 정도입니다 그렇다면 전 세계 식용유 대란의 이유는 무엇일까요? 세계적으로 가장 많이 쓰는 3대 식용유는 카놀라유 콩기름 팜류입니다. 그중에 카놀라유 최대 수출국인 캐나다와 콩기름 원료 대두의 주산지인 남미에서 지난해부터 극심한 가뭄이 이어진 탓에 작황이 부진했죠. 보통 이런 상황에서는 해바라기 시유가 대안이 되곤 하죠. 그런데 해바라기 시유 수출 1 2위 국가가 우크라이나와 러시아입니다. 두 나라는 현재 전쟁 중이죠 우크라이나는 해바라기 CU 생산량의 절반 가량을 차지하고 있기 때문에 부족 현상은 더욱 악화됐습니다. 전쟁의 여파는 남은 배 안인 황류에도 영향을 미쳤습니다. 세계 최대 판류 생산국인 인도네시아가 지난달 28일 합류 수출을 막아버렸기 때문입니다. 전쟁으로 식용유 가격이 치솟자 인도네시아 판류 업자들이 환율을 더 비싸게 팔기 위해 생산량 대부분을 수출하면서 국내분이 부족해졌고 결국 인도네시아 정부가 나서 수출 금지 조치를 단행합니다. 결국 우리나라도 우크라이나 전쟁의 여진을 겪는 것이나 마찬가지입니다. 국내 식용유 가격이 오르고 구매 개수까지 제한하자 우려의 목소리는 더욱 커지고 있는데요. 다만 국내에서는 말레이시아산 판류를 주로 사용하고 아직 가지 원재료 수급에도 큰 문제가 없다는 게 전문가 의견입니다. 게다가 인도네시아의 수출 제한 조치도 완화될 가능성이 나오는 만큼 주의를 조금 더 지켜봐야 할 것 같습니다 ']

In [3]:
b = a[0].replace('니다', '니다. ')
b = b.replace('요 ', ' 요. ')
b = b.replace('. .', '.')
b = b.replace('-(앵커)', '')
b = b.replace('-(기자)', '')
b

'한국은행은 오늘 금융통화위원회를 열고 이번 달 기준금리를 결정합니다. 현재 1.25%인 기준금리를 0.25%포인트 올릴 것이라는 전망과 동결할 것이라는 전망이 엇갈리고 있습니다. 최근 4%대의 가파른 물가 상승률과 미국 연방준비제도의 예상보다 빠른 긴축 가능성 때문에 인상할 것이라는 전망이 나옵니다. 다만 한국은행 총재가 공석인 데다 경기 침체나 이자 부담이 늘어날 우려에 동결 분위기도 감지됩니다.  '

In [4]:
text_list = b.split('.')
text_list

['한국은행은 오늘 금융통화위원회를 열고 이번 달 기준금리를 결정합니다',
 ' 현재 1',
 '25%인 기준금리를 0',
 '25%포인트 올릴 것이라는 전망과 동결할 것이라는 전망이 엇갈리고 있습니다',
 ' 최근 4%대의 가파른 물가 상승률과 미국 연방준비제도의 예상보다 빠른 긴축 가능성 때문에 인상할 것이라는 전망이 나옵니다',
 ' 다만 한국은행 총재가 공석인 데다 경기 침체나 이자 부담이 늘어날 우려에 동결 분위기도 감지됩니다',
 '  ']

In [5]:
text = b
text

'한국은행은 오늘 금융통화위원회를 열고 이번 달 기준금리를 결정합니다. 현재 1.25%인 기준금리를 0.25%포인트 올릴 것이라는 전망과 동결할 것이라는 전망이 엇갈리고 있습니다. 최근 4%대의 가파른 물가 상승률과 미국 연방준비제도의 예상보다 빠른 긴축 가능성 때문에 인상할 것이라는 전망이 나옵니다. 다만 한국은행 총재가 공석인 데다 경기 침체나 이자 부담이 늘어날 우려에 동결 분위기도 감지됩니다.  '

In [6]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [7]:
kkma = Kkma()

# text입력받아 kkma.sentences()를 이용해 문장단위로 나는 뒤 sentences로 리턴
def text2sentences(text):
    sentences= kkma.sentences(text)  # text일 때 문장별로 리스트 만듦
    for idx in range(0, len(sentences)): #길이에 따라 문장 합침
        if len(sentences[idx]) <= 10:
            sentences[idx-1] += (' ' + sentences[idx])
            sentences[idx] = ''
    return sentences

In [8]:
#기사본문 가져오기
sentences = text2sentences(text)
sentences

['한국은행은 오늘 금융통화위원회를 열고 이번 달 기준금리를 결정합니다.',
 '현재 1.25% 인 기준금리를 0.25% 포인트 올릴 것이라는 전망과 동결할 것이라는 전망이 엇갈리고 있습니다.',
 '최근 4% 대의 가파른 물가 상승률과 미국 연방준비제도의 예상보다 빠른 긴축 가능성 때문에 인상할 것이라는 전망이 나옵니다.',
 '다만 한국은행 총재가 공석인 데다',
 '경 기 침체나 이자 부담이 늘어날 우려에 동결 분위기도 감지됩니다.']

In [9]:
twitter = Twitter()
from konlpy.tag import Okt
okt = Okt()

#불용어제거
stopwords = ['( )','안녕하세요', '아', '으', '네' ,'코']
def get_nouns(sentences):
    nouns = []
    for sentence in sentences:
        if sentence != '':
            nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                  if noun not in stopwords and len(noun) > 1]))
    return nouns

In [10]:
nouns = get_nouns(sentences)

NameError: name 'twitter' is not defined

In [98]:
nouns = get_nouns(sentences)
print(sentences[0])
print(nouns[0])


지금 회부 신용 기자


# TF-IDF 모델 생성 및 그래프 생성

In [99]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

In [100]:
def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

sent_graph = build_sent_graph(nouns)
sent_graph

array([[1.        , 0.        , 0.1368084 , 0.2344078 , 0.        ,
        0.14570008, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.21462631, 0.        , 0.        ,
        0.59807016, 0.08045192, 0.        , 0.07007087, 0.07795253,
        0.        , 0.        , 0.        ],
       [0.1368084 , 0.21462631, 1.        , 0.06879399, 0.        ,
        0.16783317, 0.13297177, 0.        , 0.24369379, 0.07522276,
        0.13318705, 0.        , 0.038088  ],
       [0.2344078 , 0.        , 0.06879399, 1.        , 0.        ,
        0.07326516, 0.07455684, 0.        , 0.06493646, 0.        ,
        0.04911892, 0.        , 0.05131671],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.34743233, 0.        ,
        0.        , 0.        , 0.        ],
       [0.14570008, 0.59807016, 0.16783317, 0.07326516, 0.        ,
        1.        , 0.05893

In [101]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

In [102]:
words_graph, idx2word = build_words_graph(nouns)

In [103]:
words_graph

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [104]:
idx2word

{47: '지금',
 56: '회부',
 29: '신용',
 11: '기자',
 53: '한국은행',
 51: '총재',
 37: '인사청문회',
 50: '창용',
 57: '후보자',
 10: '금리',
 30: '신호',
 18: '물가',
 1: '가장',
 15: '때문',
 55: '환영',
 38: '인상',
 12: '기호',
 21: '보도',
 27: '성장',
 48: '질문',
 19: '미국',
 34: '오른',
 25: '선제',
 2: '강조',
 31: '악역',
 16: '마다',
 32: '얼마나',
 20: '보고',
 4: '경제',
 3: '결정',
 24: '서민이',
 17: '문제',
 41: '정부',
 49: '차원',
 54: '해법',
 13: '대출',
 42: '정책',
 23: '부동산',
 7: '관련',
 9: '규제',
 35: '완화',
 22: '부담',
 46: '주기',
 40: '점진',
 0: '가야',
 6: '과정',
 28: '소통',
 44: '조율',
 39: '입장',
 36: '윤석',
 5: '공약',
 52: '추경',
 14: '대해',
 26: '선택',
 8: '규모',
 33: '영향',
 45: '조절',
 43: '제언'}

In [105]:
for key, value in idx2word.items():
    if value == '때문':
        print(key)

15


In [106]:
def get_ranks(graph, d=0.85):
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0
        link_sum = np.sum(A[:,id])
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1
        
    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B)
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [107]:
sent_rank_idx = get_ranks(sent_graph)
sent_rank_idx

{0: 0.752643164630335,
 1: 1.3207022165881923,
 2: 1.5994902489122675,
 3: 0.9014321759198791,
 4: 0.5610996455548302,
 5: 1.4955609241323955,
 6: 0.9078667575803402,
 7: 0.15000000000000002,
 8: 1.453794768405428,
 9: 0.9001172902022545,
 10: 0.651406370244879,
 11: 0.7152666049584347,
 12: 0.7406198328707547}

In [108]:
sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)
sorted_sent_rank_idx

[2, 5, 8, 1, 6, 3, 9, 0, 12, 11, 10, 4, 7]

In [109]:
word_rank_idx = get_ranks(words_graph)
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)

# 요약하기 실행

In [126]:
llist=[]
summary = []
summarize=[]
index = []
for idx in sorted_sent_rank_idx[:5]:
    index.append(idx)
        
index.append(idx)
    
for idx in index:
    summary.append(sentences[idx])
        
for text in summary[:]:
    llist.append(text)
    
m = list(set(llist))

for i in m:
    i = i.replace('-( )', '')
    summarize.append(i)

In [127]:
summarize

' 성장이냐  새 한국은행 총재 인사청문회에서 이 창용 후보자가 금리를 올리겠다는 신호를 강하게 보냈습니다. 미국처럼 물가가 오른 뒤가 아닌 선제적으로 금리를 올려야 한다고 강조했습니다. 지금까지 사회부 신용 식 기자였습니다.'

In [112]:
summary = ' '.join(summarize)
summary

'  알겠습니다. 지금까지 사회부 신용식 기자였습니다.  새 한국은행 총재 인사청문회에서 이창용 후보자가 금리를 올리겠다는 신호를 강하게 보냈습니다. 지금은 물가 잡는 게 가장 급하기 때문에 환영받지 않더라도 금리 인상 신호를 주며 할 일을 하겠다는 겁니다. 조기호 기자의 보도입니다.  물가냐 성장이냐는 질문에 이창용 한국은행 총재 후보자는 지금은 물가부터 잡아야 할 때라는 답을 내놨습니다.  미국처럼 물가가 오른 뒤가 아닌 선제적으로 금리를 올려야 한다고 강조했습니다. 악역도 마다하지 않겠다고 했습니다.  금리 인상폭은 물가가 얼마나 오를지 보고 또 경제 성장에 해가 되지 않는 선에서 결정하겠다고 덧붙였습니다. 금리를 올렸을 때 서민이 힘들어지는 문제는 범정부 차원에서 해법을 내놔야 한다고 밝혔습니다. 또 대출 정책은 부동산과 관련돼 있고 규제를 완화하면 물가에 부담을 주기 때문에 점진적으로 가야 한다고도 했습니다. 이 과정에서 정부와 소통하고 조율을 하겠지만 정부 정책에 끌려가지는 않을 것이라는 입장도 내놨습니다.  윤석열 정부의 공약인 50조 원 추경에 대해서는 불가피한 선택이라면서도 규모는 물가에 미치는 영향을 따져서 조절해야 한다고 제언했습니다.  '

In [130]:
stop_words = ['때문', '지금', '이', '을', '의', '는', '가', '들', ',', '도',
              '은', '그', '.', '를', '너무', '고', '것' ,'로', '하는','보고','에','때','에',
              '저','하고','에서','정말','못','다','까지','또','개','수','으로','만','할',
              '과','인','님','-','임','7','봐도','하는거','그냥','해도','바로','순간',
              '잘','한','겁나','나','같이','함께','라고','쓴','아니라','아닌','옹','살','진짜','하는게','같아요','제목','적', '가야','지금',
             '관련']
    
keywords = []
index = []
for idx in sorted_word_rank_idx[:15]:
    index.append(idx)
        
for idx in index:
    keywords.append(idx2word[idx])
        
keywords = [each_word for each_word in keywords
            if each_word not in stop_words]
        
keyword = keywords[:5]

In [131]:
keyword

['물가', '금리', '정부', '정책', '인상']

def summarize(sent_num=5):
    summary = []
    index = []
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)
        
    index.append(idx)
    
    for idx in index:
        summary.append(sentences[idx])
        
    for text in summary[:]:
        print(text)